In [1]:
!pip install shap xgboost

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/82/29/923869e92c74bf07ec2b9a52ad5ac67d4184c873ba33ada7d4584356463a/shap-0.46.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for slicer==0.0.8 from https://files.pythonhosted.org/packages/63/81/9ef641ff4e12cbcca30e54e72fb0951a2ba195d0cda0ba4100e532d929db/slicer-0.0.8-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/456.1 kB ? eta -:--:--
   -- ------------------------------------ 30.7/456.1 kB 660.6 kB/s eta 0:00:01
   -------------------------- ------------- 307.2/456.1 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------- 456.1/456.1 kB 4.8 MB/s eta 0:00:00


In [42]:
train_path = r"c:/For Internships & Job Applications/personal project/Ecommerce Order & Supply Chain Dataset/archive/train"

import pandas as pd
import read_csv_files as rsv

df_dict = rsv.read_csv(train_path)

AttributeError: module 'read_csv_files' has no attribute 'read_csv'